In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import cluster, datasets
import seaborn as sns
import json
import sys
import codecs
import os
from collections import defaultdict, Counter
import pickle
from scipy.spatial.distance import cosine 

## Load the clusttering model

In [2]:
#load the desired model from ./models
modeldir_path = "../models/"
model_name = "KMEANS_glove_pmeans5_K50_R3000_F250_model.pkl" #enter model name here
clustter_model = pickle.load(open(modeldir_path+model_name, "rb"))
print(model_name+" is loaded.")


KMEANS_glove_pmeans5_K50_R3000_F250_model.pkl is loaded.


## Load the MegaDF

In [3]:
megadf_dir = "../MegaDfs/"
megadf_name = "KMEANS_glove_pmeans5_K50_R3000_F250_clustered_megadf.pkl"  #enter megadf name
megadf = pd.DataFrame()
megadf = pickle.load(open(megadf_dir+megadf_name, "rb"))

In [25]:
newabs = " ".join(megadf["abstract"].iloc[0])
print(newabs)

soft coral could drifa glomerata fam alcyoniidae capnella glomerata fam nephtheidae


## Get the list of abstracts to clustter 

In [5]:
new_abstract_list = []   #enter abstract here
newabs = input("Enter the abstract\n")


## Generate sentence embedding of the abstract

In [26]:
from basiclib.preprocessing import preproc
from basiclib.word_embedding import GloveVectors #chose the word embedding
from basiclib.sentence_embedding import get_pmeans

#load word embedding model
glovefile = "../dataset/glove.6B.50d.txt"
glove = GloveVectors(glovefile)

print("Embeding the abstract.")
newabs_embedding = get_pmeans(glove.get_embeddings(preproc(newabs))) 

print("Embedding Done!")




Loading Model
Done! Model Loaded. Vector Size is 50
Embeding the abstract.
Embedding Done!


In [27]:
print(newabs_embedding.shape)

(250,)


## Predicting the clustter using the model and finding nearest 5 papers in that clustter

In [28]:
def cos_dis(x):
    global newabs_embedding
    return cosine(x, newabs_embedding)

In [30]:
newabs_clustterlabel = clustter_model.predict([newabs_embedding,])[0]
newabs_clustter_df = megadf[megadf["clusterlabel"] == newabs_clustterlabel]
newabs_clustter_df["Dist2New"] = newabs_clustter_df["embedding"].apply(cos_dis)
newabs_clustter_df = newabs_clustter_df.sort_values(by = "Dist2New")
newabs_clustter_df.info()



<class 'pandas.core.frame.DataFrame'>
Int64Index: 49 entries, 0 to 177
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            49 non-null     int64  
 1   title         49 non-null     object 
 2   abstract      49 non-null     object 
 3   embedding     49 non-null     object 
 4   clusterlabel  49 non-null     int32  
 5   Dist2New      49 non-null     float64
dtypes: float64(1), int32(1), int64(1), object(3)
memory usage: 2.5+ KB
